In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.
    
    {chat_history}
    Human: {question}
    You: 
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=PromptTemplate.from_template(template),
)


chain.predict(question="My name is Max")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    
    Human: My name is Max
    You: 


> Finished chain.


'Hello Max! How can I assist you today?'

In [6]:
chain.predict(question="I live in Busan")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: The human introduces themselves as Max and mentions that they live in Busan. The AI greets Max and acknowledges that Busan is a beautiful city in South Korea known for its stunning beaches and vibrant culture. The AI then asks how it can assist Max.
Human: What is my name?
AI: Your name is Max.
Human: I live in Busan
AI: Yes, you mentioned earlier that you live in Busan. It's a beautiful city in South Korea known for its stunning beaches and vibrant culture. How can I assist you today?
Human: What is my name?
AI: Your name is Max.
    Human: I live in Busan
    You: 


> Finished chain.


"Yes, you mentioned earlier that you live in Busan. It's a beautiful city in South Korea known for its stunning beaches and vibrant culture. How can I assist you today?"

In [7]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.
    
    System: The human, Max, mentions that they live in Busan. The AI greets Max and acknowledges that Busan is a beautiful city in South Korea known for its stunning beaches and vibrant culture. The AI then asks how it can assist Max. Max asks what their name is, and the AI responds that their name is Max as well.
AI: Yes, you mentioned earlier that you live in Busan. It's a beautiful city in South Korea known for its stunning beaches and vibrant culture. How can I assist you today?
Human: What is my name?
AI: Your name is Max.
Human: I live in Busan
AI: Yes, you mentioned earlier that you live in Busan. It's a beautiful city in South Korea known for its stunning beaches and vibrant culture. How can I assist you today?
    Human: What is my name?
    You: 


> Finished chain.


'Your name is Max.'